In [1]:
import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

import torch
print(torch.__version__)

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams.update({'font.size': 12})

USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

1.0.1.post2


## Prioritized Replay Buffer ++

In [2]:
class PrioritizedBuffer(object):
    def __init__(self, capacity, prob_alpha=0.6):
        self.prob_alpha = prob_alpha
        self.capacity   = capacity
        self.buffer     = []
        self.pos        = 0
        self.priorities = np.zeros((capacity,), dtype=np.float32)
        self.states_count = {0:0, 1:0, 'frac': 0}
        self.states_loss = {0:0, 1:0, 'frac': 0}
    
    def push(self, state, action, reward, next_state, done):
        assert state.ndim == next_state.ndim
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
        
        max_prio = self.priorities.max() if self.buffer else 1.0
        
        if len(self.buffer) < self.capacity:
            self.buffer.append((state, action, reward, next_state, done))
        else:
            self.buffer[self.pos] = (state, action, reward, next_state, done)
        
        self.priorities[self.pos] = max_prio
        self.pos = (self.pos + 1) % self.capacity
    
    def sample(self, batch_size, beta=0.4):
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.pos]
        
        probs  = prios ** self.prob_alpha
        probs /= probs.sum()
        
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]
        
        total    = len(self.buffer)
        weights  = (total * probs[indices]) ** (-beta)
        weights /= weights.max()
        weights  = np.array(weights, dtype=np.float32)
        
        batch       = list(zip(*samples))
        states      = np.concatenate(batch[0])
        actions     = batch[1]
        rewards     = batch[2]
        next_states = np.concatenate(batch[3])
        dones       = batch[4]
        
        # increment states count
        self.states_count[0] += states.shape[0] - np.sum(states[:,-1])
        self.states_count[1] += np.sum(states[:,-1])
        
        return states, actions, rewards, next_states, dones, indices, weights
    
    def update_priorities(self, batch_indices, batch_priorities):
        for idx, prio in zip(batch_indices, batch_priorities):
            self.priorities[idx] = prio

    def __len__(self):
        return len(self.buffer)

## Cartpole Env

In [5]:
env_id = "CartPole-v0"
env = gym.make(env_id)
val_env = gym.make(env_id)
epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 500
epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

## DQN

In [6]:
class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, env.action_space.n)
        )
        
    def forward(self, x):
        return self.layers(x)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(state).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            action  = q_value.max(1)[1].data[0]
            action = int(action)
        else:
            action = random.randrange(env.action_space.n)
        return action

In [7]:
current_model = DQN(env.observation_space.shape[0] + 1, env.action_space.n)
target_model  = DQN(env.observation_space.shape[0] + 1, env.action_space.n)

if USE_CUDA:
    current_model = current_model.cuda()
    target_model  = target_model.cuda()
    
optimizer = optim.Adam(current_model.parameters())

## TD Loss

In [8]:
def compute_td_loss(batch_size, beta):
    state, action, reward, next_state, done, indices, weights = replay_buffer.sample(batch_size, beta)
    

    state      = Variable(torch.FloatTensor(np.float32(state)))
    next_state = Variable(torch.FloatTensor(np.float32(next_state)))
    action     = Variable(torch.LongTensor(action))
    reward     = Variable(torch.FloatTensor(reward))
    done       = Variable(torch.FloatTensor(done))
    weights    = Variable(torch.FloatTensor(weights))

    q_values      = current_model(state)
    next_q_values = target_model(next_state)

    q_value          = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_value     = next_q_values.max(1)[0]
    expected_q_value = reward + gamma * next_q_value * (1 - done)
    
    loss  = (q_value - expected_q_value.detach()).pow(2) * weights
    prios = loss + 1e-5
    loss  = loss.mean()
        
    optimizer.zero_grad()
    loss.backward()
    replay_buffer.update_priorities(indices, prios.data.cpu().numpy())
    optimizer.step()
    
    return loss

## Plot

In [9]:
def plot(frame_idx, rewards, losses, standard_val_rewards, noisy_val_rewards, states_count_ratios):
    clear_output(True)
    plt.figure(figsize=(20,16))
    plt.subplot(321)
    plt.title('frame %s. reward: %s' % (frame_idx, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.subplot(322)
    plt.title('loss')
    plt.plot(losses)
    plt.subplot(323)
    plt.title('frame %s. val reward on standard env: %s' % (frame_idx, np.mean(standard_val_rewards[-10:])))
    plt.plot(standard_val_rewards)
    plt.subplot(324)
    plt.title('frame %s.val reward on noisy env: %s' % (frame_idx, np.mean(noisy_val_rewards[-10:])))
    plt.plot(noisy_val_rewards)
    plt.subplot(325)
    plt.title('frame %s. proportion of selecting noisy env: %s' % (frame_idx, np.mean(states_count_ratios[-10:])))
    plt.plot(states_count_ratios)
    plt.show()

## Train

In [10]:
# Test agent on standard or noisy env
# Resulting episode reward stored in XXX_val_rewards where XXX is standard or noisy 
num_val_trials = 10
def test(noisyGame, eps):
    rewards = []
    for i in range(num_val_trials):
        epsilon = 0 
        episode_reward = 0
        state = val_env.reset()
        state = np.append(state, float(noisyGame))
        with torch.no_grad():
            while True:
                original_action = current_model.act(state, epsilon)
                if original_action != int(original_action):
                    original_action = original_action.numpy()[0]

                if noisyGame and random.uniform(0,1) < eps:
                    actual_action = 1 - original_action
                else:
                    actual_action = original_action 

                next_state, reward, done, _ = val_env.step(actual_action)
                if noisyGame:
                    reward += random.uniform(-100, 100)
                next_state = np.append(next_state, float(noisyGame))
                state = next_state
                episode_reward += reward

                if done:
                    rewards.append(episode_reward)
                    break 
    return np.mean(rewards)

In [ ]:
num_frames = 10000
batch_size = 32
gamma      = 0.99

losses = []
all_rewards = []
standard_val_rewards = [] 
noisy_val_rewards = [] 
states_count_ratios = []
episode_reward = 0

noisyGame = True
state = env.reset()
state = np.append(state, float(noisyGame))

replay_buffer = PrioritizedBuffer(100000)

# Probability of action being random in noisy state
eps = 1.

for frame_idx in range(1, num_frames + 1):
    epsilon = epsilon_by_frame(frame_idx)
    original_action = current_model.act(state, epsilon)
    
    # If in noisy environment, make action random with probability eps 
    if noisyGame and random.uniform(0,1) < eps:
        actual_action = 1 - original_action
    else:
        actual_action = original_action
    next_state, reward, done, _ = env.step(actual_action)
    
    # If in noisy environment, make reward completely random 
    if noisyGame:
        reward += random.uniform(-100, 100)
    next_state = np.append(next_state, float(noisyGame))
    replay_buffer.push(state, original_action, reward, next_state, done)

    state = next_state
    episode_reward += reward
    
    if done:
        noisyGame = 1-noisyGame
        state = env.reset()
        state = np.append(state, float(noisyGame))
        all_rewards.append(episode_reward)
        episode_reward = 0
        
    if len(replay_buffer) > batch_size:
        beta = beta_by_frame(frame_idx)
        loss = compute_td_loss(batch_size, beta)
        losses.append(loss.data.tolist())
        
    if frame_idx % 200 == 0:
        standard_val_rewards.append(test(False, eps))
        noisy_val_rewards.append(test(True, eps))
        states_count_ratios.append(float(replay_buffer.states_count[1]) / (float(replay_buffer.states_count[1])  + float(replay_buffer.states_count[0])))
        plot(frame_idx, all_rewards, losses, standard_val_rewards, noisy_val_rewards, states_count_ratios)
        
    if frame_idx % 1000 == 0:
        update_target(current_model, target_model)